<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/train_lama_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tain LaMa
This notebok runs the default training for LaMa

In [1]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Can't uninstall 'detectron2'. No files were found to uninstall.
  Running setup.py develop for detectron2


In [2]:
import os
import shutil
import glob

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# train_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/train_png.zip'
val_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
eval_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
dataset_dir = '/content/lama/my_dataset/'

In [4]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

# downloads the models for evaluations
print('Downloading resnet model for evaluation')
!mkdir -p ade20k/ade20k-resnet50dilated-ppm_deepsup/
!wget -P ade20k/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth

# see if you don't need this
# print('\n> Download the model')
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/EgqaSnLohjuzAg) -o lama-models.zip
# !unzip lama-models.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil


> Cloning the repo
Cloning into 'lama'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 283 (delta 73), reused 264 (delta 66), pack-reused 0
Receiving objects: 100% (283/283), 6.49 MiB | 37.57 MiB/s, done.
Resolving deltas: 100% (73/73), done.

> Install dependencies
     |████████████████████████████████| 12.5 MB 5.0 MB/s 
     |████████████████████████████████| 22.3 MB 70.9 MB/s 
     |████████████████████████████████| 72 kB 854 kB/s 
     |████████████████████████████████| 144 kB 92.0 MB/s 
     |████████████████████████████████| 841 kB 64.9 MB/s 
     |████████████████████████████████| 271 kB 91.8 MB/s 
     |████████████████████████████████| 47 kB 6.8 MB/s 
     |████████████████████████████████| 47.7 MB 1.1 MB/s 
     |████████████████████████████████| 948 kB 70.8 MB/s 
     |████████████████████████████████| 176 kB 81.3 MB/s 
     |████████████████████████████████| 133 kB 

In [5]:
# uses the celeb dataset
shutil.copy('/content/gdrive/MyDrive/repos/GANime/data_out/data256x256.zip', '/content/lama/data256x256.zip')
!bash fetch_data/celebahq_dataset_prepare.sh

# generate masks for test and visual_test at the end of epoch
!export PYTHONPATH=. && bash fetch_data/celebahq_gen_masks.sh

# Run training
!export TORCH_HOME=$(pwd) && export USER='matt' && export PYTHONPATH=. && python3 bin/train.py -cn lama-fourier-celeba data.batch_size=10

Epoch 0:   5% 360/6592 [02:00<34:51,  2.98it/s, loss=13.6, v_num=0]/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...
Epoch 0, global step 370: val_ssim_fid100_f1_total_mean reached 0.00000 (best 0.00000), saving model to "/content/lama/experiments/matt_2022-02-11_21-40-54_train_lama-fourier-celeba_/models/epoch=0-step=370.ckpt" as top 5
Epoch 0:   5% 360/6592 [02:10<37:34,  2.76it/s, loss=13.6, v_num=0]


In [ ]:
# val_source # 2000 or more images
# visual_test_source # 100 or more images
# eval_source # 2000 or more images

# copy the training set over
os.mkdir(dataset_dir)
shutil.copy(val_zip, dataset_dir + 'val.zip')
shutil.unpack_archive(dataset_dir + 'val.zip', dataset_dir + '/val_source/')

# makes a copy for eval and visual test
shutil.copytree(dataset_dir + 'val_source/', dataset_dir + 'eval_source/')
shutil.copytree(dataset_dir + 'val_source/', dataset_dir + 'visual_test_source/')

In [ ]:
# copies images to the training folder
img_list = glob.glob(dataset_dir + '/val_source/*')
print(f'Number of images: {len(img_list)}')
img_list = [f for f in img_list if 'mask' not in f]
print(f'Number of images: {len(img_list)}')

In [27]:
# picks 130k images and resizes them
if os.path.isdir(dataset_dir + 'train/'):
    shutil.rmtree(dataset_dir + 'train/')
os.mkdir(dataset_dir + 'train/')

import random
import cv2

for i in range(130000):
    img_file = random.choice(img_list)
    img = cv2.imread(img_file)
    img = cv2.resize(img, (256, 256))
    cv2.imwrite(dataset_dir + 'train/' + os.path.basename(img_file), img)
    
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [34]:
# increasing the celeb dataset
celeb_files = glob.glob('/content/lama/my_dataset/celeb_train/*')
print(f'Number of files {len(celeb_files)}')

# gets the max number
i = 0
while i < (130000 - len(celeb_files)):
    img_file = random.choice(celeb_files)
    shutil.copy(img_file, '/content/lama/my_dataset/celeb_train/' + str(i+130000) + '.jpg')

    i += 1

Number of files 129999


In [20]:
celeb_files = glob.glob('/content/lama/my_dataset/val/*')
print(f'Number of files {len(celeb_files)}')


Number of files 122852


In [18]:
# testing moving celeba images to training folder
celeb_files = glob.glob('/content/lama/celeba-hq-dataset/train_256/*')
print(f'Number of files {len(celeb_files)}')
shutil.copytree('/content/lama/celeba-hq-dataset/train_256', '/content/lama/my_dataset/celeb')

Number of files 26000


'/content/lama/my_dataset/celeb'

In [47]:
# makes the config file for training
!touch my_dataset.yaml
!echo "data_root_dir: $(pwd)/my_dataset/" >> my_dataset.yaml
!echo "out_root_dir: $(pwd)/experiments/" >> my_dataset.yaml
!echo "tb_dir: $(pwd)/tb_logs/" >> my_dataset.yaml
!mv my_dataset.yaml ${PWD}/configs/training/location/

# change /content/lama/configs/training/trainer/any_gpu_large_ssim_ddp_final.yaml limit_train_batches: 2600 # was 25000

In [ ]:
# tests
# shutil.rmtree('/content/lama/celeba-hq-dataset/data256x256')
# shutil.rmtree('/content/lama/celeba-hq-dataset/val_source_256')
# shutil.rmtree('/content/lama/celeba-hq-dataset/visual_test_source_256')

!export TORCH_HOME=$(pwd) && export USER='matt' && export PYTHONPATH=. && python3 bin/train.py -cn lama-fourier location=my_dataset data.batch_size=10
# !cat ${PWD}/configs/training/data/abl-04-256-mh-dist

Epoch 0:   3% 2620/94739 [14:29<8:29:37,  3.01it/s, loss=10.3, v_num=0]
Validating: 0it [00:00, ?it/s]
Validating:   0% 0/92139 [00:00<?, ?it/s]
Epoch 0:   3% 2640/94739 [14:32<8:27:05,  3.03it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2660/94739 [14:34<8:24:21,  3.04it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2680/94739 [14:36<8:21:40,  3.06it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2700/94739 [14:38<8:19:01,  3.07it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2720/94739 [14:40<8:16:24,  3.09it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2740/94739 [14:42<8:13:49,  3.10it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2760/94739 [14:44<8:11:16,  3.12it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2780/94739 [14:46<8:08:46,  3.14it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2800/94739 [14:48<8:06:18,  3.15it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2820/94739 [14:50<8:03:52,  3.17it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2840/94739 [14:52<8:01:28,  3.18it/s, loss=10.3, v_num=0]
Epoch 0:   3% 2860/94739 [14:54<7:59:06,  3.20it/s, loss=10.3, 